In [ ]:
import random
from Game import Game
from deap import base, creator, tools, algorithms
import matplotlib.pyplot as plt


if "FitnessMax" in creator.__dict__:
    del creator.FitnessMax
if "Individual" in creator.__dict__:
    del creator.Individual
